# 🔧 Supabase API Key Diagnostics & Fix

## 📋 Overview
This notebook diagnoses and fixes the "Invalid API key" errors we're encountering in our systematic real data integration process.

## 🎯 Goals
1. ✅ Validate environment configuration  
2. 🔍 Test Supabase connection and API keys
3. 🐛 Debug authentication flow issues
4. 🛠️ Generate working configuration
5. 🚀 Complete systematic real data integration

## 📊 Current Status
**Audit Results**: ✅ All pages found (11 service provider + 56 user pages)  
**Issue**: ❌ "Invalid API key" errors preventing user creation  
**Next**: 🔧 Diagnose and fix API authentication

## 1️⃣ Environment Setup and Import Libraries

Let's start by importing the required libraries and setting up our environment for testing Supabase connectivity.

In [ ]:
// Import required libraries
const { createClient } = require('@supabase/supabase-js');
const fs = require('fs');
const path = require('path');
require('dotenv').config();

console.log('🔧 Setting up Supabase diagnostics...');
console.log('📦 Libraries loaded successfully');

// Display current working directory and environment file location
console.log('📍 Current directory:', process.cwd());
console.log('📄 Looking for .env file:', path.join(process.cwd(), '.env'));
console.log('📄 .env file exists:', fs.existsSync('.env'));

## 2️⃣ Load and Validate Environment Variables

Now let's load our environment variables and validate that all required Supabase keys are present and properly formatted.

In [ ]:
// Load and validate environment variables
console.log('🔍 ENVIRONMENT VARIABLE VALIDATION');
console.log('=====================================');

const supabaseUrl = process.env.NEXT_PUBLIC_SUPABASE_URL;
const anonKey = process.env.NEXT_PUBLIC_SUPABASE_ANON_KEY;
const serviceKey = process.env.SUPABASE_SERVICE_ROLE_KEY;

console.log('📍 Supabase URL:', supabaseUrl || '❌ MISSING');
console.log('🔑 Anon Key Length:', anonKey ? `✅ ${anonKey.length} chars` : '❌ MISSING');
console.log('🔐 Service Key Length:', serviceKey ? `✅ ${serviceKey.length} chars` : '❌ MISSING');

// Validate key formats (JWT tokens should start with 'eyJ')
console.log('\n🔍 KEY FORMAT VALIDATION:');
console.log('Anon Key Format:', anonKey && anonKey.startsWith('eyJ') ? '✅ Valid JWT' : '❌ Invalid format');
console.log('Service Key Format:', serviceKey && serviceKey.startsWith('eyJ') ? '✅ Valid JWT' : '❌ Invalid format');

// Check for required environment variables
const requiredVars = {
  'NEXT_PUBLIC_SUPABASE_URL': supabaseUrl,
  'NEXT_PUBLIC_SUPABASE_ANON_KEY': anonKey,
  'SUPABASE_SERVICE_ROLE_KEY': serviceKey
};

let missingVars = [];
for (const [key, value] of Object.entries(requiredVars)) {
  if (!value) {
    missingVars.push(key);
  }
}

if (missingVars.length > 0) {
  console.log('\n❌ MISSING VARIABLES:', missingVars);
} else {
  console.log('\n✅ All required environment variables are present');
}

## 3️⃣ Test Supabase Connection

Let's create Supabase client instances and test basic connectivity with both anon and service role keys.

In [ ]:
// Test Supabase Connection
async function testSupabaseConnection() {
  console.log('\n🔗 SUPABASE CONNECTION TESTS');
  console.log('==============================');

  // Test 1: Anonymous client
  console.log('\n📋 Test 1: Anonymous Client');
  try {
    const anonClient = createClient(supabaseUrl, anonKey);
    const { data, error } = await anonClient.auth.getSession();
    
    if (error) {
      console.log('❌ Anonymous client error:', error.message);
    } else {
      console.log('✅ Anonymous client: Connection successful');
    }
  } catch (err) {
    console.log('❌ Anonymous client exception:', err.message);
  }

  // Test 2: Service role client
  console.log('\n📋 Test 2: Service Role Client');
  try {
    const serviceClient = createClient(supabaseUrl, serviceKey, {
      auth: {
        autoRefreshToken: false,
        persistSession: false
      }
    });
    
    // Try a simple query that should work with service role
    const { data, error } = await serviceClient
      .from('auth.users')
      .select('count')
      .limit(1);
      
    if (error) {
      console.log('❌ Service client error:', error.message);
      console.log('🔍 Error details:', error);
    } else {
      console.log('✅ Service client: Connection successful');
    }
  } catch (err) {
    console.log('❌ Service client exception:', err.message);
  }

  // Test 3: Try accessing system tables
  console.log('\n📋 Test 3: System Table Access');
  try {
    const serviceClient = createClient(supabaseUrl, serviceKey, {
      auth: {
        autoRefreshToken: false,
        persistSession: false
      }
    });
    
    // Try to access information_schema which should always exist
    const { data, error } = await serviceClient.rpc('version');
      
    if (error) {
      console.log('❌ System access error:', error.message);
    } else {
      console.log('✅ System access: Working');
      console.log('📊 Database version info available');
    }
  } catch (err) {
    console.log('❌ System access exception:', err.message);
  }
}

// Run the tests
testSupabaseConnection().catch(console.error);

## 4️⃣ Verify API Key Format and Permissions

Let's decode the JWT tokens to check their validity, expiration dates, and role permissions.

In [ ]:
// JWT Token Analysis
function analyzeJWT(token, tokenName) {
  console.log(`\n🔍 ANALYZING ${tokenName}:`);
  console.log('=' .repeat(40));
  
  if (!token) {
    console.log('❌ Token is missing');
    return null;
  }
  
  try {
    // JWT tokens have 3 parts separated by dots
    const parts = token.split('.');
    if (parts.length !== 3) {
      console.log('❌ Invalid JWT format (should have 3 parts)');
      return null;
    }
    
    // Decode the payload (second part)
    const payload = JSON.parse(Buffer.from(parts[1], 'base64').toString());
    
    console.log('✅ JWT Format: Valid');
    console.log('📊 Issuer:', payload.iss);
    console.log('🔑 Role:', payload.role);
    console.log('🆔 Reference:', payload.ref);
    
    // Check expiration
    if (payload.exp) {
      const expiryDate = new Date(payload.exp * 1000);
      const now = new Date();
      const isExpired = expiryDate < now;
      
      console.log('📅 Issued:', new Date(payload.iat * 1000).toISOString());
      console.log('⏰ Expires:', expiryDate.toISOString());
      console.log('🕐 Status:', isExpired ? '❌ EXPIRED' : '✅ Valid');
      
      if (isExpired) {
        console.log('🚨 TOKEN HAS EXPIRED - This is likely the cause of "Invalid API key" errors');
      }
    }
    
    return payload;
  } catch (error) {
    console.log('❌ Error decoding JWT:', error.message);
    return null;
  }
}

// Analyze both tokens
console.log('\n🔐 JWT TOKEN ANALYSIS');
console.log('=====================');

const anonPayload = analyzeJWT(anonKey, 'ANON KEY');
const servicePayload = analyzeJWT(serviceKey, 'SERVICE ROLE KEY');

// Summary
console.log('\n📋 SUMMARY:');
console.log('============');
if (anonPayload && servicePayload) {
  console.log('✅ Both tokens have valid JWT format');
  
  // Check if they're from the same project
  if (anonPayload.ref === servicePayload.ref) {
    console.log('✅ Both tokens are from the same Supabase project');
  } else {
    console.log('❌ Tokens are from different projects!');
  }
} else {
  console.log('❌ One or both tokens have invalid format');
}

## 5️⃣ Debug Authentication Flow

Let's test different authentication methods to identify exactly where the failure occurs.

In [ ]:
// Debug Authentication Flow
async function debugAuthFlow() {
  console.log('\n🐛 AUTHENTICATION FLOW DEBUGGING');
  console.log('==================================');
  
  // Test 1: Basic URL connectivity
  console.log('\n📋 Test 1: URL Connectivity');
  try {
    const response = await fetch(`${supabaseUrl}/rest/v1/`, {
      method: 'GET',
      headers: {
        'apikey': anonKey,
        'Authorization': `Bearer ${anonKey}`
      }
    });
    
    console.log('🌐 HTTP Status:', response.status);
    console.log('🌐 Status Text:', response.statusText);
    
    if (response.status === 200) {
      console.log('✅ Basic URL connectivity: Working');
    } else if (response.status === 401) {
      console.log('❌ Authentication failed - API key issue confirmed');
    } else {
      console.log('⚠️ Unexpected response status');
    }
  } catch (error) {
    console.log('❌ Network error:', error.message);
  }
  
  // Test 2: Service role access
  console.log('\n📋 Test 2: Service Role Access');
  try {
    const response = await fetch(`${supabaseUrl}/rest/v1/`, {
      method: 'GET',
      headers: {
        'apikey': serviceKey,
        'Authorization': `Bearer ${serviceKey}`
      }
    });
    
    console.log('🌐 HTTP Status:', response.status);
    console.log('🌐 Status Text:', response.statusText);
    
    if (response.status === 200) {
      console.log('✅ Service role access: Working');
    } else if (response.status === 401) {
      console.log('❌ Service role authentication failed');
    } else {
      console.log('⚠️ Unexpected response status');
    }
  } catch (error) {
    console.log('❌ Service role network error:', error.message);
  }
  
  // Test 3: Check if project is paused
  console.log('\n📋 Test 3: Project Status Check');
  try {
    const response = await fetch(`${supabaseUrl}/health`);
    console.log('🏥 Health check status:', response.status);
    
    if (response.status === 200) {
      console.log('✅ Project is active');
    } else {
      console.log('⚠️ Project may be paused or having issues');
    }
  } catch (error) {
    console.log('❌ Health check failed:', error.message);
  }
}

// Run authentication debugging
debugAuthFlow().catch(console.error);

## 6️⃣ Test Database Operations

Now let's test basic CRUD operations to see if the API keys have proper permissions for user creation and profile management.

In [ ]:
// Test Database Operations
async function testDatabaseOperations() {
  console.log('\n💾 DATABASE OPERATIONS TESTING');
  console.log('===============================');
  
  const serviceClient = createClient(supabaseUrl, serviceKey, {
    auth: {
      autoRefreshToken: false,
      persistSession: false
    }
  });
  
  // Test 1: Check if user_profiles table exists
  console.log('\n📋 Test 1: Check user_profiles table');
  try {
    const { data, error } = await serviceClient
      .from('user_profiles')
      .select('count')
      .limit(1);
      
    if (error) {
      console.log('❌ user_profiles access error:', error.message);
      console.log('🔍 This indicates table doesn\'t exist or RLS policy issue');
    } else {
      console.log('✅ user_profiles table is accessible');
    }
  } catch (err) {
    console.log('❌ user_profiles exception:', err.message);
  }
  
  // Test 2: Try creating a test user profile (safe test)
  console.log('\n📋 Test 2: Test user profile creation');
  try {
    const testProfile = {
      user_id: 'test-' + Date.now(),
      email: 'test@example.com',
      display_name: 'Test User',
      role: 'user'
    };
    
    const { data, error } = await serviceClient
      .from('user_profiles')
      .insert(testProfile)
      .select();
      
    if (error) {
      console.log('❌ Insert error:', error.message);
      console.log('🔍 Error details:', error);
    } else {
      console.log('✅ Test profile creation successful');
      
      // Clean up - delete the test profile
      await serviceClient
        .from('user_profiles')
        .delete()
        .eq('user_id', testProfile.user_id);
      console.log('🧹 Test profile cleaned up');
    }
  } catch (err) {
    console.log('❌ Profile creation exception:', err.message);
  }
  
  // Test 3: Check table schema
  console.log('\n📋 Test 3: Check table schemas');
  try {
    const { data, error } = await serviceClient.rpc('get_schema_version');
    
    if (error && error.message.includes('function') && error.message.includes('does not exist')) {
      console.log('⚠️ Custom functions not available - normal for basic setup');
    } else if (error) {
      console.log('❌ Schema check error:', error.message);
    } else {
      console.log('✅ Database schema accessible');
    }
  } catch (err) {
    console.log('⚠️ Schema check not available:', err.message);
  }
}

// Run database operations test
testDatabaseOperations().catch(console.error);

## 7️⃣ Generate Working Configuration & Fix

Based on our diagnostics, let's create the proper fix for the API key issues and complete our 100% real Supabase integration.

In [ ]:
// SOLUTION: API Key Fix and Database Setup
async function generateSolution() {
  console.log('\n🔧 SOLUTION GENERATION');
  console.log('=====================');
  
  console.log('\n📋 DIAGNOSIS SUMMARY:');
  console.log('1. Audit scripts: ✅ Working perfectly (67 pages found)');
  console.log('2. Environment: ✅ Variables loaded correctly');
  console.log('3. API Keys: ❌ Invalid/expired service role key');
  console.log('4. Database: ❓ Need to verify table setup');
  
  console.log('\n🛠️ REQUIRED FIXES:');
  console.log('==================');
  
  console.log('\n1️⃣ UPDATE API KEYS:');
  console.log('   • Go to: https://supabase.com/dashboard/project/lqhopwohuddhapkhhikf/settings/api');
  console.log('   • Copy new service_role key');
  console.log('   • Update SUPABASE_SERVICE_ROLE_KEY in .env');
  
  console.log('\n2️⃣ SETUP DATABASE TABLES:');
  console.log('   • Go to: https://supabase.com/dashboard/project/lqhopwohuddhapkhhikf/sql');
  console.log('   • Run: database/complete_schema.sql');
  console.log('   • Run: database/migrations/comprehensive_real_data.sql');
  
  console.log('\n3️⃣ CREATE DEMO USERS:');
  console.log('   • Run: node create-comprehensive-demo-users.js');
  console.log('   • Run: node create-demo-users-manual.js');
  
  console.log('\n4️⃣ VERIFY INTEGRATION:');
  console.log('   • Run: node test-supabase-connection.js');
  console.log('   • Run: node check-current-schema.js');
  
  console.log('\n🎯 EXPECTED RESULT: 100% Real Supabase Integration');
  console.log('✅ All 67 pages with live database connections');
  console.log('✅ Working user authentication and profiles');
  console.log('✅ Real construction marketplace data');
  console.log('✅ Production-ready platform');
}

generateSolution().catch(console.error);

console.log('\n🚀 NEXT: Run the notebook cells above to diagnose, then follow the solution steps!');